# Towards Visually Explaining Variational Autoencoders
To run this notebook, activate the conda environment via the environment.yml file and start jupyter notebook within this environmnent. 

In [23]:
import random

## MNIST Anomaly Detection

In [21]:
run -t code/test_expVAE.py --dataset=mnist --model=vanilla --batch_size=32 --model_path='./ckpt/vanilla_best.pth' --no_auroc --target='encoder.2'

using device cuda
layer issss encoder.2


In [25]:
img = random.choice([x for x in os.listdir("test_results_mnist_vanilla")
               if os.path.isfile(os.path.join("test_results_mnist_vanilla", x))])
print(img)

430-7-attmap.png


## USCD-Ped1 Anomaly Detection

In [20]:
run code/test_expVAE.py --dataset=ucsd-ped --model=vanilla_ped1 --batch_size=32 --model_path='./ckpt/vanilla_ped1_best.pth' --target='encoder.2'

using device cuda


UnboundLocalError: local variable 'test_dataset' referenced before assignment

## MVTec Anomaly Detection

In [ ]:
run code/test_expVAE.py --dataset=mvtec --model=resnet18 --batch_size=2 --model_path='./ckpt/resnet18_best.pth' --target='?'

## dSprite Latent Space Disentanglement